## Gauss Elimination
Given equation

$$Ax=b$$

like

$$
\begin{bmatrix}
6 & -2 & 2 & 4\\
12 & -8 & 6 & 10\\
3 & -13 & 9 & 3\\
-6 & 4 & 1 & -18\\
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
x_3\\
x_4
\end{bmatrix}
=
\begin{bmatrix}
12\\
34\\
27\\
-38
\end{bmatrix}
$$

the Gauss elimination turns it to an equivalent (ie having the same solution) equation

$$Ux=b^\star$$

where

$$
U =
\begin{bmatrix}
u_1_1 & u_1_2 & \dots & u_1_n \\
0 & u_2_2  & \dots & u_2_n \\
\vdots & \vdots   & \ddots &  \vdots & \\
0& 0 & \dots & u_n_n\\
\end{bmatrix}
$$
In the above example it will be
$$
\begin{bmatrix}
6 & -2 & 2 & 4\\
0 & -4 & 2 & 2\\
0 & 0 & 2 & -5\\
0 & 0 & 0 & -3\\
\end{bmatrix}
\begin{bmatrix}
x_1\\
x_2\\
x_3\\
x_4
\end{bmatrix}
=
\begin{bmatrix}
12\\
10\\
-9\\
-3
\end{bmatrix}
$$
which is very easy to solve with a backward substitution: $x_4= 1$, $2x_3-5\cdot1 = -9$ so $x_3=-2$ etc.


In [1]:
import notebooks.latex_utils as latex
import numpy as np


A = np.array(
    [
        [ 6., -2, 2, 4 ],
        [ 12, -8, 6, 10 ],
        [ 3, -13, 9, 3 ],
        [ -6, 4,  1, -18 ]
    ]
)
# A = np.array(
#     [
#         [ 2., 3, -6 ],
#         [ 1, -6, 8],
#         [ 3, -2, 1]
#     ]
# )

# U = A.copy()
# U[2,1] = 0.5
# b = np.array(
#     [ 12., 34, 27 ]
# )
b = np.array(
    [ 12., 34, 27, -38 ]
)

def minor(A, i, j):
    return np.delete(np.delete(A, i, 0), j, 1)

def gauss(A: np.ndarray, b: np.ndarray):
    """
    Gaussian elimination
    :param A: Matrix with coefficients of a linear equation set
    :param b: Vector of free constants ('right side')
    :return: (L, U, b1) where A = LU and b1 is a transformed vector of the right side
    """
    n, _ = np.shape(A)
    U = A.copy()
    L = np.diagflat([1. for _ in range(n)])
    b1 = b.copy()
    for i in range(n): # main row
        main_element = U[i, i]
        for j in range(i+1, n):  # rows
            multiplier = U[j, i] / main_element
            for k in range(i, n):  # columns
                U[j, k] = U[j, k] - U[i, k] * multiplier
            b1[j] = b1[j] - b1[i] * multiplier
            L[j, i] = multiplier
    return L, U, b1

def scaled_gauss(A: np.ndarray, b: np.ndarray):
    """
    Gaussian elimination with row scaling
    :param A: Matrix with coefficients of a linear equation set
    :param b: Vector of free constants ('right side')
    :return: (P, L, U, b1) where PA = LU and b1 is a transformed vector of the right side
    """
    n, _ = np.shape(A)
    U = A.copy()
    p = np.arange(n)
    L = np.zeros(shape = (n, n))
    s = [ np.max(np.abs(row)) for row in A ]
    b1 = b.copy()
    for i in range(n-1): # main row
        r = [ np.abs(U[:, i][p[l]]) / s[p[l]] for l in range(i, n) ]
        swapi = np.argmax(r) + i
        temp = p[i]
        p[i] = p[swapi]
        p[swapi] = temp
        main_element = U[p[i], i]
        for j in range(i+1, n):  # rows
            pj = p[j]
            multiplier = U[pj, i] / main_element
            for k in range(i, n):  # columns
                U[pj, k] = U[pj, k] - U[p[i], k] * multiplier
            b1[pj] = b1[pj] - b1[p[i]] * multiplier
            L[pj, i] = multiplier
    P = np.zeros(shape = (n, n))
    for i in range(n):
        P[i, p[i]] = 1
    L = P@L + np.diagflat([1 for _ in range(n)])
    return P, L, P@U, b1

def solve_backward_substitution(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solves a linear equation on assumption that A is an upper triangular matrix
    :param A: an upper triangular matrix of coefficients
    :param b: free variables
    :return: Solution vector
    """
    n, _ = np.shape(A)
    x = np.ndarray(shape = n, dtype='float')
    x[n - 1] =  b[n-1]/A[n-1, n-1]
    for i in range(n-2, -1, -1):
        summ = np.sum([A[i,j] * x[j] for j in range(i+1, n)])
        x[i] = 1. / A[i,i] * (b[i] - summ)
    return x

def solve_forward_substitution(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solves a linear equation on assumption that A is a lower triangular matrix
    :param A: a lower triangular matrix of coefficients
    :param b: free variables
    :return: Solution vector
    """
    return solve_backward_substitution(A.transpose(), b)

latex.printlatex("Ax = b")

P, L, U, b1 = scaled_gauss(A, b)
x = solve_backward_substitution(U, b1)
latex.printlatex("x = " + latex.latex_matrix(np.array([x])))
latex.printlatex("L = " + latex.latex_matrix(L))
latex.printlatex("U = " + latex.latex_matrix(U))
latex.printlatex("P = " + latex.latex_matrix(P))
latex.printlatex("A = " + latex.latex_matrix(A))

latex.printlatex("PA = " + latex.latex_matrix(P @ A))
latex.printlatex("LU = " + latex.latex_matrix(L @ U))


# printlatex(latex_matrix(U) + r"\cdot " + latex_matrix(np.array([x]).transpose()) + r" =" + latex_matrix(np.array([b1]).transpose()))
# printlatex("A=LU=" + latex_matrix(L) + r"\cdot " +  latex_matrix(U))


$$\begin{align}Ax = b\end{align}$$

$$\begin{align}x = \begin{bmatrix}-46.96314102564103&111.08173076923079&160.47115384615387&48.75000000000001\\\end{bmatrix}\end{align}$$

$$\begin{align}L = \begin{bmatrix}1.0&0.0&0.0&0.0\\0.5&1.0&0.0&0.0\\-1.0&-0.16666666666666666&1.0&0.0\\2.0&0.3333333333333333&-0.15384615384615383&1.0\\\end{bmatrix}\end{align}$$

$$\begin{align}U = \begin{bmatrix}6.0&-2.0&2.0&4.0\\0.0&-12.0&8.0&1.0\\0.0&0.0&4.333333333333333&-13.833333333333334\\0.0&0.0&0.0&-0.46153846153846145\\\end{bmatrix}\end{align}$$

$$\begin{align}P = \begin{bmatrix}1.0&0.0&0.0&0.0\\0.0&0.0&1.0&0.0\\0.0&0.0&0.0&1.0\\0.0&1.0&0.0&0.0\\\end{bmatrix}\end{align}$$

$$\begin{align}A = \begin{bmatrix}6.0&-2.0&2.0&4.0\\12.0&-8.0&6.0&10.0\\3.0&-13.0&9.0&3.0\\-6.0&4.0&1.0&-18.0\\\end{bmatrix}\end{align}$$

$$\begin{align}PA = \begin{bmatrix}6.0&-2.0&2.0&4.0\\3.0&-13.0&9.0&3.0\\-6.0&4.0&1.0&-18.0\\12.0&-8.0&6.0&10.0\\\end{bmatrix}\end{align}$$

$$\begin{align}LU = \begin{bmatrix}6.0&-2.0&2.0&4.0\\3.0&-13.0&9.0&3.0\\-6.0&4.0&1.0&-18.0\\12.0&-8.0&5.999999999999999&10.0\\\end{bmatrix}\end{align}$$